In [2]:
import os
import pandas as pd
dataset_path = '../data/nutrition5k_reconstructed/'

dataset_path = '../../data/nutrition5k_reconstructed/'

image_path = os.path.join(dataset_path, 'images')
label_path = os.path.join(dataset_path, 'labels/labels.csv')
train_path = os.path.join(dataset_path, 'metadata/train_ids.csv')
test_path = os.path.join(dataset_path, 'metadata/test_ids.csv')

# Training Files
with open(train_path, 'r') as f:
    train_files = f.read().splitlines()
    print('Number of training files:', len(train_files))
    
# Testing Files
with open(test_path, 'r') as f:
    test_files = f.read().splitlines()
    print('Number of testing files:', len(test_files))

# Labels
labels = pd.read_csv(label_path)
print('Number of labels:', len(labels))
print('\n------------------------------------\n')

# Displaying the first 5 rows of the labels
print(labels.head())


Number of training files: 2755
Number of testing files: 507
Number of labels: 3262

------------------------------------

                id    calories   mass        fat       carb    protein
0  dish_1566851198  463.047455  342.0  21.119635  42.903687  25.897209
1  dish_1565119439  465.412231  189.0  19.563423  42.858475  27.351355
2  dish_1561753432  380.321686  238.0  19.463066  16.882177  33.369026
3  dish_1561662054  419.438782  292.0  23.838249  26.351543  25.910593
4  dish_1563205982  316.239502  469.0   4.363000  54.792000  16.148998


In [2]:
import numpy as np
import pandas as pd

# Normalizing outputs in range [0-1]
calories_max = labels.iloc[:, 1].max()
mass_max = labels.iloc[:, 2].max()
fat_max = labels.iloc[:, 3].max()
carb_max = labels.iloc[:, 4].max()
protein_max = labels.iloc[:, 5].max()

labels.iloc[:, 1] = labels.iloc[:, 1] / calories_max
labels.iloc[:, 2] = labels.iloc[:, 2] / mass_max
labels.iloc[:, 3] = labels.iloc[:, 3] / fat_max
labels.iloc[:, 4] = labels.iloc[:, 4] / carb_max
labels.iloc[:, 5] = labels.iloc[:, 5] / protein_max

outputs_max_values = {'max_calorie': [calories_max], 'max_mass': [mass_max], 
                      'max_fat': [fat_max], 'max_carb': [carb_max], 'max_protein': [protein_max]} 
df_outputs_max_values = pd.DataFrame(outputs_max_values)
df_outputs_max_values.to_csv("./outputs_max_values.csv", index=False)
print("Outputs MAX values: \n", df_outputs_max_values)

print('\n------------------------------------\n')
print('Normalized labels:')
print(labels.head())

Outputs MAX values: 
    max_calorie  max_mass     max_fat    max_carb  max_protein
0  3943.325195    3051.0  106.343002  844.568604   147.491821

------------------------------------

Normalized labels:
                id  calories      mass       fat      carb   protein
0  dish_1566851198  0.117426  0.112094  0.198599  0.050800  0.175584
1  dish_1565119439  0.118025  0.061947  0.183965  0.050746  0.185443
2  dish_1561753432  0.096447  0.078007  0.183022  0.019989  0.226243
3  dish_1561662054  0.106367  0.095706  0.224164  0.031201  0.175675
4  dish_1563205982  0.080196  0.153720  0.041028  0.064876  0.109491


In [ ]:
# Splitting the data into training, validation and testing
from sklearn.model_selection import train_test_split
import shutil

# Create Training Dataframe
train_labels = labels[labels['id'].isin(train_files)]
print('Number of training labels:', len(train_labels))

# Create Testing Dataframe
test_labels = labels[labels['id'].isin(test_files)]

# Splitting the test data into validation and testing
test_labels, val_labels = train_test_split(test_files, test_size=0.4, random_state=42)
print('Number of validation labels:', len(val_labels))
print('Number of testing labels:', len(test_labels))
# convert the validation and testing labels to dataframes
val_labels = labels[labels['id'].isin(val_labels)]
test_labels = labels[labels['id'].isin(test_labels)]

# Saving the training, validation and testing labels
base_dir = './data'
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)
    
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

train_labels.to_csv(os.path.join(base_dir, 'train_labels.csv'), index=False)
val_labels.to_csv(os.path.join(base_dir, 'val_labels.csv'), index=False)
test_labels.to_csv(os.path.join(base_dir, 'test_labels.csv'), index=False)

Number of training labels: 2755
Number of validation labels: 203
Number of testing labels: 304


In [ ]:
# Speical Task about predicting based on the image and masses.
labels = pd.read_csv(label_path)
mass_inputs = labels.drop(columns=['calories', 'fat', 'carb', 'protein'])
mass_inputs.to_csv(os.path.join(base_dir, 'mass_inputs.csv'), index=False)